In [2]:
import json
import py4j
import boto3
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

# import recordlinkage
import pandas as pd
import re
import hashlib
import textdistance
from abydos.phonetic import SpanishMetaphone
from unidecode import unidecode
import networkx
import networkx as nx
from networkx.algorithms.components.connected import connected_components
from pyspark import SparkContext, SparkConf
import ibis
import random
import string
from pyspark.sql.functions import udf
spark = SparkSession.builder.appName("ConnectedComponents").getOrCreate()
# loaded_dob_graph = nx.read_graphml("dob_1edit.graphml")


your 131072x1 screen size is bogus. expect trouble


23/08/25 13:19:07 WARN Utils: Your hostname, DESKTOP-0SFECE9 resolves to a loopback address: 127.0.1.1; using 172.18.64.40 instead (on interface eth0)
23/08/25 13:19:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/25 13:19:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
def pcolumn_hashing(pFirstName, pLastName):
    return hashlib.md5(
        str(pFirstName).encode("utf-8")
        + str(pLastName).encode("utf-8")
        # + str(dob).encode("utf-8")
    ).hexdigest()


# df = spark.read.csv("big_cluster.csv")
df = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("big_cluster.csv")
)
df = df.withColumn(
    "group_hash",
    F.udf(pcolumn_hashing, T.StringType())(
        # F.col("pFirstName"), F.col("pLastName"), F.col("dob_hash")
        F.col("pfirstname"),
        F.col("plastname"),
        # F.col("dob_hash"),
    ),
)
# df = df.withColumn("dob_count", F.size(F.col("DateOfBirth")))
# df_single = df.filter(F.col("dob_count") == 1)
# df = df.filter(F.col("dob_count") > 1)

df = df.fillna("FNU", subset="firstname")
replacement_expr = " junior alias son of $1"
df = df.withColumn(
    "firstname",
    F.regexp_replace(F.col("FirstName"), "^(jr|jr.)", " junior alias son of $1"),
)

print("start ", df.count())

df.createOrReplaceTempView("df_dedupe")
print(
    "num of dob nulls ",
    spark.sql("select count(*) as cnt from df_dedupe where dateofbirth = 'CCAI_NULL'").show(),
)
df.createOrReplaceTempView("df_dedupe")
df = spark.sql(
    """
select firstname, lastname, passengerid, pfirstname,plastname, provisionalprimarykey, group_hash,
case when dateofbirth = 'CCAI_NULL' then null else dateofbirth end as dateofbirth from df_dedupe
"""
)


dob_null_handler = """
SELECT
    t1.firstname,
    t1.lastname,
    t1.passengerid,
    t1.pfirstname,
    t1.plastname,
    t1.group_hash,
    t1.provisionalprimarykey,
    COALESCE(t1.dateofbirth, t2.dateofbirth) AS dateofbirth
FROM
    df_dedupe t1
left join (
    select firstname, lastname, dateofbirth from (
        select firstname, lastname, dateofbirth from df_dedupe where dateofbirth is not null and firstname in (select firstname from df_dedupe where dateofbirth == 'CCAI_NULL') and lastname in (select lastname from df_dedupe where dateofbirth == 'CCAI_NULL')
        ) group by firstname, lastname, dateofbirth
    ) as t2 on t1.firstname = t2.firstname and t1.lastname = t2.lastname
"""
# df.createOrReplaceTempView("df_dedupe")
# df = spark.sql(dob_null_handler)
from pyspark.sql.window import Window
window_spec = Window.partitionBy(["firstname","lastname"])
df = df.withColumn("dateofbirthfilled", F.max("dateofbirth").over(window_spec))
df = df.withColumn("dateofbirth", F.when(F.col("dateofbirth").isNull(),F.col("dateofbirthfilled")).otherwise(F.col("dateofbirth")))
# print("dob nulls handled ", df.count())

drop_nulls = """
select * from df_dedupe where dateofbirth is not NULL
"""
df.createOrReplaceTempView("df_dedupe")
df = spark.sql(drop_nulls)
print("dropping unresolved dob nulls ", df.count())

groupby = """
SELECT
    firstname,
    lastname,
    dateofbirth,
    first(group_hash) as group_hash,
    ARRAY_AGG(provisionalprimarykey) AS passengerid_list,
    first(passengerid) as passengerid,
    first(provisionalprimarykey) as provisionalprimarykey,
    first(pfirstname) as pfirstname,
    first(plastname) as plastname
FROM
    df_dedupe
GROUP BY
    firstname,
    lastname,
    dateofbirth;
"""
df.createOrReplaceTempView("df_dedupe")
df = spark.sql(groupby)
print("groupby to reduce exact duplicates ", df.count())

crossjoin = """
select t1.*,
t2.firstname as firstname_right, 
t2.lastname as lastname_right, 
t2.dateofbirth as dateofbirth_right, 
t2.passengerid as passengerid_right, 
t2.provisionalprimarykey as provisionalprimarykey_right,
t2.passengerid_list as passengerid_list_right, 
t2.group_hash as group_hash_right 
from df_dedupe as t1 left join df_dedupe as t2 on t1.dateofbirth == t2.dateofbirth and t1.passengerid != t2.passengerid and t1.pfirstname == t2.pfirstname and t1.plastname == t2.plastname
"""
df.createOrReplaceTempView("df_dedupe")
df = spark.sql(crossjoin)
print("cross join ", df.count())
df.show()

import ceja

df = df.withColumn(
    "firstnamesim",
    F.when(
        (F.col("firstname") != "CCAI_NULL") & (F.col("firstname_right") != "CCAI_NULL"),
        ceja.jaro_winkler_similarity(F.col("firstname"), F.col("firstname_right")),
    ).otherwise(0),
)
df = df.withColumn(
    "lastnamesim",
    F.when(
        (F.col("lastname") != "CCAI_NULL") & (F.col("lastname_right") != "CCAI_NULL"),
        ceja.jaro_winkler_similarity(F.col("lastname"), F.col("lastname_right")),
    ).otherwise(0),
)


# condition = (F.col("firstnamesim") >= 0.85) & (F.col("lastnamesim") >= 0.90)
# df = df.filter(condition)
df = df.withColumn(
    "passengerid_list",
    F.when(
        (F.col("firstnamesim") >= 0.85) & (F.col("lastnamesim") >= 0.90),
        F.concat(F.col("passengerid_list"), F.col("passengerid_list_right")),
    ).otherwise(F.col("passengerid_list")),
)
print("condition based concat ", df.count())

# df = df.select("firstname")
# df.write.csv("big_cluster_filter.csv")


def cc_nx_udf(df: pd.DataFrame) -> pd.DataFrame:
    group_hash = df["group_hash"].iloc[0]
    dob_tag_list = df["passengerid_list"].to_list()

    def to_graph(l):
        G = networkx.Graph()
        for part in l:
            # each sublist is a bunch of nodes
            G.add_nodes_from(part)
            # it also imlies a number of edges:
            G.add_edges_from(to_edges(part))
        return G

    def to_edges(l):
        """
        treat `l` as a Graph and returns it's edges
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
        """
        it = iter(l)
        last = next(it)

        for current in it:
            yield last, current
            last = current

    G = to_graph(dob_tag_list)
    cluster = [list(x) for x in list(connected_components(G))]
    dob = []
    dob_hash = []
    gp_list = []
    for ind, each in enumerate(cluster):
        for each_element in each:
            dob.append(each_element)
            gp_list.append(group_hash)
            dob_hash.append(hashlib.md5(str(each[0]).encode("utf-8")).hexdigest())
    result_df = pd.DataFrame(
        {
            "group_hash": gp_list,
            "passenger_hash": dob_hash,
            "provisionalprimarykey": dob,
        }
    )
    return result_df


ccSchema = T.StructType(
    [
        T.StructField("group_hash", T.StringType()),
        T.StructField("provisionalprimarykey", T.StringType()),
        T.StructField("passenger_hash", T.StringType()),
    ]
)
df = df.groupby(["pfirstname", "plastname"]).applyInPandas(cc_nx_udf, schema=ccSchema)
print("final count ", df.count())
df.show()
df.write.csv("deduped.csv")

start  22111
+---+
|cnt|
+---+
| 23|
+---+

num of dob nulls  None
dropping unresolved dob nulls  22106
groupby to reduce exact duplicates  11989


cross join  33396


+---------+--------+-----------+--------------------+--------------------+-----------+---------------------+----------+---------+---------------+--------------+-----------------+-----------------+---------------------------+----------------------+--------------------+
|firstname|lastname|dateofbirth|          group_hash|    passengerid_list|passengerid|provisionalprimarykey|pfirstname|plastname|firstname_right|lastname_right|dateofbirth_right|passengerid_right|provisionalprimarykey_right|passengerid_list_right|    group_hash_right|
+---------+--------+-----------+--------------------+--------------------+-----------+---------------------+----------+---------+---------------+--------------+-----------------+-----------------+---------------------------+----------------------+--------------------+
|     yang|   huang| 1980-01-11|33abaee4f25af4990...| [349556516_9487190]|  349556516|    349556516_9487190|       YNG|      UNG|      yung feng|         huang|       1980-01-11|        3278722

final count  22106


+--------------------+---------------------+--------------------+
|          group_hash|provisionalprimarykey|      passenger_hash|
+--------------------+---------------------+--------------------+
|33abaee4f25af4990...|    349556516_9487190|9686d0a681a2f371f...|
|33abaee4f25af4990...|    350746051_9487190|bbf44b0b63eadd2bc...|
|33abaee4f25af4990...|    349574278_9487190|bbf44b0b63eadd2bc...|
|33abaee4f25af4990...|    349610605_9487190|794b621441f352b73...|
|33abaee4f25af4990...|    349194873_9487190|794b621441f352b73...|
|33abaee4f25af4990...|    352960646_9487190|b8696198edee258d6...|
|33abaee4f25af4990...|    348364708_9487190|b8696198edee258d6...|
|33abaee4f25af4990...|    351279715_9487190|6f844ca59971ec290...|
|33abaee4f25af4990...|    348072442_9487190|6f844ca59971ec290...|
|33abaee4f25af4990...|    350725823_9487190|3258b5630eac4379a...|
|33abaee4f25af4990...|    350159663_9487190|3258b5630eac4379a...|
|33abaee4f25af4990...|    348000951_9487190|787bd070b1dbfc29f...|
|33abaee4f

AnalysisException: path file:/home/anorak/CeBu-CDP-pyspark/deduped.csv already exists.

In [13]:
def pcolumn_hashing(pFirstName, pLastName):
    return hashlib.md5(
        str(pFirstName).encode("utf-8")
        + str(pLastName).encode("utf-8")
        # + str(dob).encode("utf-8")
    ).hexdigest()


# df = spark.read.csv("big_cluster.csv")
df = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("big_cluster.csv")
)
df = df.withColumn(
    "group_hash",
    F.udf(pcolumn_hashing, T.StringType())(
        # F.col("pFirstName"), F.col("pLastName"), F.col("dob_hash")
        F.col("pfirstname"),
        F.col("plastname"),
        # F.col("dob_hash"),
    ),
)
# df = df.withColumn("dob_count", F.size(F.col("DateOfBirth")))
# df_single = df.filter(F.col("dob_count") == 1)
# df = df.filter(F.col("dob_count") > 1)

df = df.fillna("FNU", subset="firstname")
replacement_expr = " junior alias son of $1"
df = df.withColumn(
    "firstname",
    F.regexp_replace(F.col("FirstName"), "^(jr|jr.)", " junior alias son of $1"),
)

print("start ", df.count())

df.createOrReplaceTempView("df_dedupe")
print(
    "num of dob nulls ",
    spark.sql("select count(*) as cnt from df_dedupe where dateofbirth = 'CCAI_NULL'").show(),
)
df.createOrReplaceTempView("df_dedupe")
df = spark.sql(
    """
select firstname, lastname, passengerid, pfirstname,plastname, provisionalprimarykey, group_hash,
case when dateofbirth = 'CCAI_NULL' then null else dateofbirth end as dateofbirth from df_dedupe
"""
)

df.createOrReplaceTempView("df_dedupe")
spark.sql("select count(*) from df_dedupe where dateofbirth is null").show()


# dob_null_handler = """
# SELECT
#     firstname,
#     lastname,
#     COALESCE(dateofbirth, last(dateofbirth) over (partition by firstname, lastname order by dateofbirth)) AS dateofbirth
# FROM
#     df_dedupe

# """

# df.createOrReplaceTempView("df_dedupe")
# df = spark.sql(dob_null_handler)
from pyspark.sql.window import Window
window_spec = Window.partitionBy(["firstname","lastname"])
df = df.withColumn("dateofbirthfilled", F.max("dateofbirth").over(window_spec))
df = df.withColumn("dateofbirth", F.when(F.col("dateofbirth").isNull(),F.col("dateofbirthfilled")).otherwise(F.col("dateofbirth")))


drop_nulls = """
select * from df_dedupe where dateofbirth is NULL
"""
df.createOrReplaceTempView("df_dedupe")
df = spark.sql(drop_nulls)
print("nulls dropped",df.count())
df.show()


start  22111
+---+
|cnt|
+---+
| 23|
+---+

num of dob nulls  None
+--------+
|count(1)|
+--------+
|      23|
+--------+

+-----------------+--------+-----------+----------+---------+---------------------+--------------------+-----------+-----------------+
|        firstname|lastname|passengerid|pfirstname|plastname|provisionalprimarykey|          group_hash|dateofbirth|dateofbirthfilled|
+-----------------+--------+-----------+----------+---------+---------------------+--------------------+-----------+-----------------+
|ying fan lawrence|    wong|  350544265|       YNG|      UNG|   350544265_11696517|33abaee4f25af4990...|       null|       1964-12-10|
|ying fan lawrence|    wong|  351838415|       YNG|      UNG|   351838415_11696517|33abaee4f25af4990...|       null|       1964-12-10|
|ying fan lawrence|    wong|  352895648|       YNG|      UNG|   352895648_11696517|33abaee4f25af4990...|       null|       1964-12-10|
|ying fan lawrence|    wong|  350514253|       YNG|      UNG|   350

+----------+--------+-----------+----------+---------+---------------------+--------------------+-----------+-----------------+
| firstname|lastname|passengerid|pfirstname|plastname|provisionalprimarykey|          group_hash|dateofbirth|dateofbirthfilled|
+----------+--------+-----------+----------+---------+---------------------+--------------------+-----------+-----------------+
|yang kwang|    wong|  326835508|       YNG|      UNG|    326835508_9699408|33abaee4f25af4990...|       null|             null|
|  ying hao|    hung|  342631254|       YNG|      UNG|   342631254_11391188|33abaee4f25af4990...|       null|             null|
|  ying hao|    hung|  343521301|       YNG|      UNG|   343521301_11391188|33abaee4f25af4990...|       null|             null|
|ying khuen|    wong|  303763842|       YNG|      UNG|    303763842_7358045|33abaee4f25af4990...|       null|             null|
|   yung ho|    wong|  354412250|       YNG|      UNG|   354412250_11816718|33abaee4f25af4990...|       

In [8]:

def cc_nx(df: pd.DataFrame) -> pd.DataFrame:
    # print("SHAPE",df.shape,df.columns)
    gp = df["group_hash"].iloc[0]
    phd = {
        "ProvisionalPrimaryKey": df["ProvisionalPrimaryKey"].iloc[0],
        "passenger_hash": hashlib.md5(
            str(df["ProvisionalPrimaryKey"].iloc[0]).encode("utf-8")
        ).hexdigest(),
    }
    if df.shape[0] == 1:
        return pd.DataFrame(
            {
                "group_hash": [df["group_hash"].iloc[0]],
                "passenger_hash_dict": [json.dumps(phd)],
            }
        )

    ibis.duckdb.connect()
    ibis.options.interactive = True
    df = ibis.memtable(df, name="df")
    from ibis import _

    # get all ccai_null dob rows
    df_dob_null = df[df.DateOfBirth == "CCAI_NULL"]
    for inndx, each_null_row in df_dob_null.to_pandas().iterrows():
        # find if any row has the same firstname and lastname
        # print(df.columns)
        df_row = df[
            (df.DateOfBirth != "CCAI_NULL")
            & (df.FirstName == each_null_row["FirstName"])
            & (df.LastName == each_null_row["LastName"])
        ].to_pandas()
        if df_row.shape[0] > 0:
            cond = df.ProvisionalPrimaryKey == each_null_row["ProvisionalPrimaryKey"]
            dob_modified = cond.ifelse(df_row.iloc[0]["DateOfBirth"], df.DateOfBirth)
            # print("replacing", dob_modified)
            df = df.mutate(DateOfBirth=dob_modified)
        else:
            # print("removing")
            df = df[df.ProvisionalPrimaryKey != each_null_row["ProvisionalPrimaryKey"]]

    df = df.group_by(["FirstName", "LastName", "DateOfBirth"]).agg(
        PassengerID=_.PassengerID.first(),
        # FirstName=_.FirstName.first(),
        # LastName=_.LastName.first(),
        # DateOfBirth=_.DateOfBirth.first(),
        Gender=_.Gender.first(),
        exact_match_groups=_.ProvisionalPrimaryKey.collect(),
        ProvisionalPrimaryKey=_.ProvisionalPrimaryKey.first(),
    )
    df = df.to_pandas()
    exact_match_groups = df["exact_match_groups"].to_list()
    # print("here2", df.shape)

    # dob1 = []
    # dob2 = []
    # id_ = []
    # for index, each_row in df.iterrows():
    #     for each_dob in list(loaded_dob_graph.neighbors(each_row["DateOfBirth"])) + [each_row["DateOfBirth"]]:
    #         id_.append(each_row["ProvisionalPrimaryKey"])
    #         dob1.append(each_row["DateOfBirth"])
    #         dob2.append(each_dob)
    # # print(len(id_),len(dob1),len(dob2))
    # right_joiner = pd.DataFrame({
    #     "right_ProvisionalPrimaryKey": id_,
    #     "right_DateOfBirth": dob1,
    #     "right_exploded_DateOfBirth": dob2,
    # })
    # # print("right joiner",right_joiner.shape)
    df = ibis.memtable(df, name="left_df")
    # right_joiner = ibis.memtable(right_joiner,name="right_joiner")
    # df1 = df.join(right_joiner, (df.ProvisionalPrimaryKey != right_joiner.right_ProvisionalPrimaryKey) & (df.DateOfBirth == right_joiner.right_DateOfBirth)).view()
    # # print("join1 done")
    # print(df1.to_pandas())
    # df2 = df1.join(df, (df1.right_exploded_DateOfBirth == df.DateOfBirth) & (df1.ProvisionalPrimaryKey != df.ProvisionalPrimaryKey))
    # df = df2.to_pandas()
    # print("joins done",df.shape)
    df1 = df.view()
    df = df.join(df1, ((df.PassengerID != df1.PassengerID) & (df.DateOfBirth == df1.DateOfBirth))) 
    df = df.to_pandas()
    # id_ = []
    # fn = []
    # ln = []
    # dob = []
    # g = []
    # ppk = []
    # id_r = []
    # fn_r = []
    # ln_r = []
    # dob_r = []
    # g_r = []
    # ppk_r = []
    # for index, each_row in df.iterrows():
    #     print(index,len(list(loaded_dob_graph.neighbors(each_row["DateOfBirth"])) + [each_row["DateOfBirth"]]))
    #     for each_dob in list(loaded_dob_graph.neighbors(each_row["DateOfBirth"])) + [
    #         each_row["DateOfBirth"]
    #     ]:
    #         df_filtered = df[df["DateOfBirth"] == each_dob]
    #         # print(each_dob, each_row["DateOfBirth"], df_filtered.shape)
    #         if df_filtered.shape[0] != 0:
    #             for inndex, each_dub in df_filtered.iterrows():
    #                 if each_row["PassengerID"] != each_dub["PassengerID"]:
    #                     id_.append(each_row["PassengerID"])
    #                     fn.append(each_row["FirstName"])
    #                     ln.append(each_row["LastName"])
    #                     dob.append(each_row["DateOfBirth"])
    #                     g.append(each_row["Gender"])
    #                     ppk.append(each_row["ProvisionalPrimaryKey"])

    #                     id_r.append(each_dub["PassengerID"])
    #                     fn_r.append(each_dub["FirstName"])
    #                     ln_r.append(each_dub["LastName"])
    #                     dob_r.append(each_dub["DateOfBirth"])
    #                     g_r.append(each_dub["Gender"])
    #                     ppk_r.append(each_dub["ProvisionalPrimaryKey"])
    # print("here3",len(id_))
    
    # df = pd.DataFrame(
    #     {
    #         "PassengerID": id_,
    #         "FirstName": fn,
    #         "LastName": ln,
    #         "DateOfBirth": dob,
    #         "Gender": g,
    #         "ProvisionalPrimaryKey": ppk,
    #         "PassengerID_right": id_r,
    #         "FirstName_right": fn_r,
    #         "LastName_right": ln_r,
    #         "DateOfBirth_right": dob_r,
    #         "Gender_right": g_r,
    #         "ProvisionalPrimaryKey_right": ppk_r,
    #     }
    # )
    if df.shape[0] > 0:
        # print("here",df.shape,df)
        df_ = ibis.memtable(df, name="df1")

        from ibis import udf

        @udf.scalar.python
        def jaro_winkler_udf(a: str, b: str) -> float:
            return textdistance.jaro_winkler.similarity(a, b)

        @udf.scalar.python
        def damerau_levelstien_udf(a: str, b: str) -> float:
            return textdistance.damerau_levenshtein.distance(a, b)
        # print(df_.to_pandas())
        FirstNameSim = (
            ibis.case()
            .when(
                ((df_.FirstName != "CCAI_NULL") & (df_.FirstName_right != "CCAI_NULL")),
                jaro_winkler_udf(
                    df_.FirstName.split(" ")[0], df_.FirstName_right.split(" ")[0]
                ),
            )
            .else_(0)
            .end()
        )
        LastNameSim = (
            ibis.case()
            .when(
                ((df_.LastName != "CCAI_NULL") & (df_.LastName_right != "CCAI_NULL")),
                jaro_winkler_udf(df_.LastName, df_.LastName_right),
            )
            .else_(0)
            .end()
        )

        DateOfBirthSim = (
            ibis.case()
            .when(
                ((df_.DateOfBirth != "CCAI_NULL") & (df_.DateOfBirth_right != "CCAI_NULL")),
                damerau_levelstien_udf(df_.DateOfBirth, df_.DateOfBirth_right),
            )
            .else_(0)
            .end()
        )
        # GenderSim = (
        #     ibis.case()
        #     .when(
        #         (
        #             (df_.Gender != "CCAI_NULL")
        #             & (df_.Gender_right != "CCAI_NULL")
        #             & (df_.Gender == df_.Gender_right)
        #         ),
        #         1,
        #     )
        #     .else_(0)
        #     .end()
        # )
        df_ = df_.mutate(FirstNameSim=FirstNameSim)
        df_ = df_.mutate(LastNameSim=LastNameSim)
        df_ = df_.mutate(DateOfBirthSim=DateOfBirthSim)
        # df_ = df_.mutate(GenderSim=GenderSim)
        condition1 = (
            (df_.FirstNameSim >= 0.85) & (df_.LastNameSim >= 0.90) 
            # & (df_.GenderSim == 1)
        )
        df_ = df_[condition1]

        condition2 = df_.DateOfBirthSim <= 2
        condition3 = (df_.DateOfBirth == "CCAI_NULL") | (df_.DateOfBirth_right == "CCAI_NULL")
        condition4 = (df_.FirstName.split(" ")[0] == df_.FirstName_right.split(" ")[0]) & (
            df_.LastName == df_.LastName_right
        )
        df_ = df_[(condition2 | (condition3 & condition4))]
        df_ = df_.to_pandas()
        if df_.shape[0] == 0:
            matches_list = []
        else:
            matches_list = (
                df_[["ProvisionalPrimaryKey", "ProvisionalPrimaryKey_right"]].to_numpy().tolist()
            )
    else:
        matches_list = []

    matches_list = matches_list + exact_match_groups

    def to_graph(l):
        G = networkx.Graph()
        for part in l:
            # each sublist is a bunch of nodes
            G.add_nodes_from(part)
            # it also imlies a number of edges:
            G.add_edges_from(to_edges(part))
        return G

    def to_edges(l):
        """
        treat `l` as a Graph and returns it's edges
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
        """
        it = iter(l)
        last = next(it)

        for current in it:
            yield last, current
            last = current

    G = to_graph(matches_list)
    cluster = [list(x) for x in list(connected_components(G))]
    # arr1 = []
    # arr2 = []
    gps = []
    cluster_dict = []
    for ind, each in enumerate(cluster):
        for each_element in each:
            # arr1.append(each_element)
            # arr2.append(hashlib.md5(str(each[0]).encode("utf-8")).hexdigest())
            phd = {
                "ProvisionalPrimaryKey": each_element,
                "passenger_hash": hashlib.md5(str(each[0]).encode("utf-8")).hexdigest(),
            }
            # phd = "1"
            gps.append(gp)
            cluster_dict.append(json.dumps(phd))
            # cluster_dict.append(phd)
    # print(cluster_dict)
    return pd.DataFrame({"group_hash": gps, "passenger_hash_dict": cluster_dict})

df = pd.read_csv("big_cluster.csv")
df["group_hash"] = "12345678dsfg"
df = df.rename(
    columns={
        "provisionalprimarykey": "ProvisionalPrimaryKey",
        "firstname": "FirstName",
        "lastname": "LastName",
        "dateofbirth": "DateOfBirth",
        "gender": "Gender",
        "passengerid": "PassengerID",
    }
)
print(df[df["DateOfBirth"] == "CCAI_NULL"].shape)
# df = df[:10000]
cc_nx(df)

(23, 9)
here (11916, 14)         FirstName LastName DateOfBirth  PassengerID  Gender  \
0            yang     wang  1989-02-20    343291692       1   
1      ying chuan     wang  1995-04-10    344183786       2   
2            ying     wang  1990-08-28    339361082       2   
3            ying     wang  1990-08-27    333898923       2   
4            yang     wang  1989-06-21    306039537       1   
...           ...      ...         ...          ...     ...   
11911        ying     weng  1993-05-11    348785008       1   
11912        yong     wang  1996-08-18    352826206       1   
11913        ying    huang  1981-07-16    351445547       1   
11914        ying    huang  1988-05-21    348457334       1   
11915        ying    huang  1988-05-21    348457334       1   

                                      exact_match_groups  \
0      [343291692_10810884, 349538730_9487190, 348995...   
1                                   [344183786_10810884]   
2                 [339361082_8269290, 

,group_hash,passenger_hash_dict
0,12345678dsfg,"{""ProvisionalPrimaryKey"": ""353246436_9487190"",..."
1,12345678dsfg,"{""ProvisionalPrimaryKey"": ""348597992_9487190"",..."
2,12345678dsfg,"{""ProvisionalPrimaryKey"": ""351540479_9487190"",..."
3,12345678dsfg,"{""ProvisionalPrimaryKey"": ""348494143_9487190"",..."
4,12345678dsfg,"{""ProvisionalPrimaryKey"": ""353293844_9487190"",..."
...,...,...
9992,12345678dsfg,"{""ProvisionalPrimaryKey"": ""353529208_9487190"",..."
9993,12345678dsfg,"{""ProvisionalPrimaryKey"": ""348142679_9487190"",..."
9994,12345678dsfg,"{""ProvisionalPrimaryKey"": ""348330512_9487190"",..."
9995,12345678dsfg,"{""ProvisionalPrimaryKey"": ""331218118_9487190"",..."


In [15]:
import networkx as nx
from itertools import combinations

def calculate_edit_distance(str1, str2):
    return sum(c1 != c2 for c1, c2 in zip(str1, str2))

def build_graph(date_of_births):
    lookup = {}
    graph = nx.Graph()
    graph.add_nodes_from(date_of_births)
    
    for dob1, dob2 in combinations(date_of_births, 2):
        
        if calculate_edit_distance(dob1, dob2) == 1:
            graph.add_edge(dob1, dob2)
    
    return graph

def get_edge_relations(graph, dob):
    return list(graph.neighbors(dob))

date_of_births = [
"2008-06-20",
"1995-12-16",
"1983-10-20",
"1990-01-03",
"1975-11-16",
"1989-04-19",
"1973-05-07",
"1970-12-15",
"1975-04-03",
]

import pandas as pd
date_of_births = pd.read_csv("dedup_debug/dob_list.csv")["dateofbirth"].to_list()
graph = build_graph(date_of_births)

dob_to_check = pd.read_csv("dedup_debug/dob_list.csv")["dateofbirth"].to_list()[0]
related_dobs = get_edge_relations(graph, dob_to_check)

print(f"Date of Births related to {dob_to_check}: {related_dobs}")
nx.write_graphml(graph, "dob_1edit.graphml")


In [6]:
import networkx as nx
from itertools import combinations
import pandas as pd
loaded_graph = nx.read_graphml("dob_1edit.graphml")

def calculate_edit_distance(str1, str2):
    return sum(c1 != c2 for c1, c2 in zip(str1, str2))

date_of_births = pd.read_csv("dedup_debug/dob_list.csv")["dateofbirth"].to_list()
new_dobs = ["2023-11-01","2023-11-02","2023-10-01"] 
# + date_of_births
print(list(set(new_dobs) - set(date_of_births)))

loaded_graph.add_nodes_from(new_dobs)

for each_new in new_dobs:
    for each in list(loaded_graph.nodes()):
        if calculate_edit_distance(each_new, each) == 1:
            loaded_graph.add_edge(each_new, each)
            print("adding ", each_new, each)


['2023-10-01', '2023-11-02', '2023-11-01']
adding  2023-11-01 2022-11-01
adding  2023-11-01 2023-01-01
adding  2023-11-01 2021-11-01
adding  2023-11-01 2003-11-01
adding  2023-11-01 2013-11-01
adding  2023-11-01 2020-11-01
adding  2023-11-01 2023-11-02
adding  2023-11-01 2023-10-01
adding  2023-11-02 2022-11-02
adding  2023-11-02 2003-11-02
adding  2023-11-02 2020-11-02
adding  2023-11-02 2023-01-02
adding  2023-11-02 2013-11-02
adding  2023-11-02 2021-11-02
adding  2023-11-02 2023-11-01
adding  2023-10-01 2020-10-01
adding  2023-10-01 2003-10-01
adding  2023-10-01 2013-10-01
adding  2023-10-01 2021-10-01
adding  2023-10-01 2022-10-01
adding  2023-10-01 2023-11-01


In [66]:
for i in pd.read_csv("dedup_debug/dob_list.csv")["dateofbirth"].to_list():
    if i =="Non":
        print("!!!!")

In [ ]:
dob_to_check = pd.read_csv("dedup_debug/dob_list.csv")["dateofbirth"].to_list()[0]
related_dobs = get_edge_relations(graph, dob_to_check)

print(f"Date of Births related to {dob_to_check}: {related_dobs}")

In [ ]:
list(graph.neighbors(dob))

In [63]:
import ibis
import pandas as pd

data = pd.DataFrame({
    'PassengerID': [1, 2, 3, 4, 5],
    'FirstName': ['A', 'B', 'C', 'D', 'E'],
    'LastName': ['A', 'B', 'C', 'D', 'E'],
    'DateOfBirth': ['A', 'B', 'C', 'D', 'E'],
    'Gender': ['A', 'B', 'C', 'D', 'E'],
    'ProvisionalPrimaryKey': ['A', 'B', 'C', 'D', 'E']
})

mapping = {
    "A":["A","B"],
    "B":["C","B"],
    "C":["C","D"],
    "D":["E","D"],
    "E":[],
}
id_ = []
fn = []
ln = []
dob = []
g = []
ppk = []
id_r = []
fn_r = []
ln_r = []
dob_r = []
g_r = []
ppk_r = []
for index, each_row in data.iterrows():
    for each_dob in list(graph.neighbors(each_row["DateOfBirth"]))+[each_row["DateOfBirth"]]:
        df_filtered = data[data["DateOfBirth"] == each_dob]
        if df_filtered.shape[0] != 0:
            for inndex, each_dub in df_filtered.iterrows():
                if each_row["PassengerID"] != each_dub["PassengerID"]:
                    id_.append(each_row["PassengerID"])
                    fn.append(each_row["FirstName"])
                    ln.append(each_row["LastName"])
                    dob.append(each_row["DateOfBirth"])
                    g.append(each_row["Gender"])
                    ppk.append(each_row["ProvisionalPrimaryKey"])

                    id_r.append(each_dub["PassengerID"])
                    fn_r.append(each_dub["FirstName"])
                    ln_r.append(each_dub["LastName"])
                    dob_r.append(each_dub["DateOfBirth"])
                    g_r.append(each_dub["Gender"])
                    ppk_r.append(each_dub["ProvisionalPrimaryKey"])
data_joined = pd.DataFrame({
    'PassengerID': id_,
    'FirstName': fn,
    'LastName': ln,
    'DateOfBirth': dob,
    'Gender': g,
    'ProvisionalPrimaryKey': ppk,
    'PassengerID_right': id_r,
    'FirstName_right': fn_r,
    'LastName_right': ln_r,
    'DateOfBirth_right': dob_r,
    'Gender_right': g_r,
    'ProvisionalPrimaryKey_right': ppk_r,
})
data_joined

,PassengerID,FirstName,LastName,DateOfBirth,Gender,ProvisionalPrimaryKey,PassengerID_right,FirstName_right,LastName_right,DateOfBirth_right,Gender_right,ProvisionalPrimaryKey_right
0,1,A,A,A,A,A,1,A,A,A,A,A
1,1,A,A,A,A,A,2,B,B,B,B,B
2,1,A,A,A,A,A,1,A,A,A,A,A
3,2,B,B,B,B,B,3,C,C,C,C,C
4,2,B,B,B,B,B,2,B,B,B,B,B
5,2,B,B,B,B,B,2,B,B,B,B,B
6,3,C,C,C,C,C,3,C,C,C,C,C
7,3,C,C,C,C,C,4,D,D,D,D,D
8,3,C,C,C,C,C,3,C,C,C,C,C
9,4,D,D,D,D,D,5,E,E,E,E,E


In [60]:
for each in data.iterrows():
    print(each)

PassengerID
FirstName
LastName
DateOfBirth
Gender
ProvisionalPrimaryKey


In [34]:
import networkx as nx

class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

def insert(root, word):
    node = root
    for char in word:
        if char not in node.children:
            node.children[char] = TrieNode()
        node = node.children[char]
    node.is_end_of_word = True

def build_graph_optimized(date_of_births):
    root = TrieNode()
    graph = nx.Graph()

    for dob in date_of_births:
        insert(root, dob)

    for dob in date_of_births:
        node = root
        for i, char in enumerate(dob):
            for candidate_char in range(ord(char) - 1, ord(char) + 2):
                if chr(candidate_char) in node.children:
                    related_dob = dob[:i] + chr(candidate_char) + dob[i+1:]
                    if related_dob != dob and related_dob in date_of_births:
                        graph.add_edge(dob, related_dob)
            if char in node.children:
                node = node.children[char]
            else:
                break

    return graph

def get_edge_relations(graph, dob):
    return list(graph.neighbors(dob))

# Example usage
import pandas as pd

date_of_births = pd.read_csv("dedup_debug/dob_list.csv")["dateofbirth"].to_list()[:2000]
graph = build_graph_optimized(date_of_births)

dob_to_check = date_of_births[0]
related_dobs = get_edge_relations(graph, dob_to_check)

print(f"Date of Births related to {dob_to_check}: {related_dobs}")


Date of Births related to 2008-06-20: ['2008-05-20', '2008-06-30']


In [36]:
import networkx as nx
from itertools import combinations

def calculate_edit_distance(str1, str2):
    # Use the built-in difflib library to calculate the edit distance
    from difflib import ndiff
    return sum(1 for diff in ndiff(str1, str2) if diff[0] != ' ')

def build_graph(date_of_births):
    graph = nx.Graph()
    graph.add_nodes_from(date_of_births)
    
    # Create a dictionary to store the already calculated edit distances
    edit_distance_cache = {}
    
    for i, dob1 in enumerate(date_of_births):
        for dob2 in date_of_births[i+1:]:
            if (dob1, dob2) not in edit_distance_cache:
                edit_distance = calculate_edit_distance(dob1, dob2)
                edit_distance_cache[(dob1, dob2)] = edit_distance
            else:
                edit_distance = edit_distance_cache[(dob1, dob2)]
            
            if edit_distance == 1:
                graph.add_edge(dob1, dob2)
    
    return graph

def get_edge_relations(graph, dob):
    return list(graph.neighbors(dob))

import pandas as pd

date_of_births = pd.read_csv("dedup_debug/dob_list.csv")["dateofbirth"].to_list()[:2000]
graph = build_graph(date_of_births)
dob_to_check = date_of_births[0]
related_dobs = get_edge_relations(graph, dob_to_check)
print(f"Date of Births related to {dob_to_check}: {related_dobs}")

In [6]:
import textdistance
str1 = "christina janell"
str2 = "christensen janaya"
str1 = "christian"
str2 = "christian junior alias son of christian"
str1 = "john derrick"
str2 = "john mark"
# str1 = "rick john"
# str2 = "rick john fredrich"
# str1 = "christianne"
# str2 = "christine alfred"
print(textdistance.jaro(str1, str2))
print(textdistance.jaro_winkler(str1, str2))

0.7870370370370371
0.8722222222222222


In [4]:
import textdistance
dob1 = "1996-10-05"
dob2 = "1991-10-06"
# calculate dam_levenshtein similarity
print(textdistance.damerau_levenshtein.normalized_similarity(dob1, dob2))


0.8


In [89]:
import json
import py4j
import boto3
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import recordlinkage
import pandas as pd
import re
import hashlib
from abydos.phonetic import SpanishMetaphone
from unidecode import unidecode
import networkx 
from networkx.algorithms.components.connected import connected_components

def dedupe(id_arr,first_arr, last_arr,date_arr, email_arr, phone_arr, gender_arr):
    # return id_arr
    print("UDF called")
    # if len(id_arr) == 1 or len(id_arr) > 5000:
    if len(id_arr) == 1:
        print("UDF called 1")
        resHash = hashlib.md5(str(id_arr[0]).encode("utf-8")).hexdigest()
        return [resHash for i in range(len(id_arr))]
        # return [id_arr[0]]
    master_dict ={}
    for index, each in enumerate(id_arr):
        master_dict[each] = {
            "ProvisionalPrimaryKey":each,
            "FirstName":first_arr[index],
            "LastName":last_arr[index],
            "DateOfBirth":date_arr[index],
            # "EmailAddress":email_arr[index],
            # "Phone":phone_arr[index],
            "Gender":gender_arr[index]
        }
    
    # create dataframe
    df = pd.DataFrame(master_dict).T
    # fill nulls in ProvisionalPrimaryKey with ccai_null
    df['ProvisionalPrimaryKey'] = df['ProvisionalPrimaryKey'].fillna('ccai_null')
    df['FirstName'] = df['FirstName'].fillna('FNU')
    # replace Gender with value 0 with None
    df["Gender"] = df["Gender"].replace(0, None)
    print("UDF called 2")
    # remove index
    df = df.reset_index(drop=True)
    # get groups of lists of ProvisionalPrimaryKey that has the same FirstName, LastName, DateOfBirth
    list_of_lists = []
    df_temp_gp = df.groupby(["FirstName","LastName", "DateOfBirth"])
    list_of_lists = df_temp_gp["ProvisionalPrimaryKey"].apply(list).to_list()
    print("!!!!!!!!!!!!!!")
    print(list_of_lists)
    print(df_temp_gp.first().reset_index())
    # list_of_lists = []
    df = df_temp_gp.first().reset_index() 
    df = df.replace('ccai_null', None)

    
    if df.shape[0]>1:
        # make ProvisionalPrimaryKey as index
        df = df.set_index('ProvisionalPrimaryKey')
        df['FirstName'] = df['FirstName'].apply(lambda x: re.sub(r' jr|\.jr', ' junior alias son of '+x, x))
        # convert DateOfBirth to datetime
        df['DateOfBirth'] = pd.to_datetime(df['DateOfBirth'], format='%Y-%m-%d')
        # convert it to string format
        df['DateOfBirth'] = df['DateOfBirth'].dt.strftime('%Y-%m-%d')
        indexer = recordlinkage.Index()
        indexer.full()
        # indexer.add(recordlinkage.index.SortedNeighbourhood(left_on="FirstName",window=501))
        
        candidate_links = indexer.index(df)
        print ("UDF called 3")

        compare_cl = recordlinkage.Compare()
        # compare_cl.exact("FirstName", "FirstName", label="FirstName")
        compare_cl.string("FirstName", "FirstName", method="jarowinkler", threshold=0.85, label="FirstName")
        compare_cl.string("LastName", "LastName", method="jarowinkler", threshold=0.90, label="LastName")
        compare_cl.string("DateOfBirth", "DateOfBirth", method="damerau_levenshtein", threshold=0.8, label="DateOfBirth")
        # compare_cl.date("DateOfBirth", "DateOfBirth", label="DateOfBirth",swap_month_day=1)
        # compare_cl.string("EmailAddress", "EmailAddress", label="EmailAddress", threshold=0.95)
        # compare_cl.string("Phone", "Phone", label="Phone", threshold=0.95)
        compare_cl.exact("Gender", "Gender", label="Gender")

        features = compare_cl.compute(candidate_links, df)
        print ("UDF called 4")
        # iterate over df columns
        df.fillna('ccai_null', inplace=True)
        # make index as ProvisionalPrimaryKey column
        df = df.reset_index()
        matches_list = []
        # iterate on each row
        for index, row in features.iterrows():            
            if row["FirstName"] == 1 and row["LastName"] == 1 and row["Gender"] == 1:
                if row["DateOfBirth"] == 1:
                    matches_list.append(list(index))
                elif str(df[df["ProvisionalPrimaryKey"]== index[0]]["DateOfBirth"].values[0]) == 'ccai_null' or str(df[df["ProvisionalPrimaryKey"]== index[1]]["DateOfBirth"].values[0]) == 'ccai_null':
                    if str(df[df["ProvisionalPrimaryKey"]== index[0]]["FirstName"].values[0]) == str(df[df["ProvisionalPrimaryKey"]== index[1]]["FirstName"].values[0]) and str(df[df["ProvisionalPrimaryKey"]== index[0]]["LastName"].values[0]) == str(df[df["ProvisionalPrimaryKey"]== index[1]]["LastName"].values[0]):
                        matches_list.append(list(index))

        # for col in df.columns:
        #     if col in features.columns:
        #         if df[col].value_counts(normalize=True).iloc[0] >= 0.8:
        #             features = features.drop(col, axis=1)
                
        # print(features.columns)
        # print(features)
        # print("threshold", features.shape[1]*0.72)
        # matches = features[features.sum(axis=1) >= features.shape[1]*0.72]
        # matches_list = matches.index.tolist()
        # matches_list = [list(x) for x in matches_list]
        matches_list = matches_list + list_of_lists
    else:
        print("else")
        matches_list = list_of_lists
    print(matches_list)
    def to_graph(l):
        G = networkx.Graph()
        for part in l:
            # each sublist is a bunch of nodes
            G.add_nodes_from(part)
            # it also imlies a number of edges:
            G.add_edges_from(to_edges(part))
        return G

    def to_edges(l):
        """ 
            treat `l` as a Graph and returns it's edges 
            to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
        """
        it = iter(l)
        last = next(it)

        for current in it:
            yield last, current
            last = current    

    G = to_graph(matches_list)
    cluster = [list(x) for x in list(connected_components(G))]
    print ("UDF called 4")

    return_hash_arr = []
    for i in range(len(id_arr)):
        bool_check = False
        for each in cluster:
            if id_arr[i] in each:
                return_hash_arr.append(hashlib.md5(str(each[0]).encode("utf-8")).hexdigest())
                # return_hash_arr.append(each[0])
                bool_check = True
                break
        if bool_check == False:
            return_hash_arr.append(hashlib.md5(str(id_arr[i]).encode("utf-8")).hexdigest())
            # return_hash_arr.append(id_arr[i])
    print ("UDF called 5")
    # return return_hash_arr
    # create a dataframe with return_hash_arr, first_arr
    df_return = pd.DataFrame({"return":return_hash_arr,"firstname":first_arr, "dob":date_arr,"gender":gender_arr})
    df_return = df_return.sort_values(by=['return'])
    return df_return

# def dedupe(id_arr,first_arr, last_arr,date_arr, email_arr, phone_arr, gender_arr):

dedupe([1,2,3,4],["candice jennifer","candice","candice jennifer","candice jennifer"], ["iyog","iyog","iyog","iyog"],["ccai_null","1973-11-16","1973-11-16","1973-11-16"],["ccai_null","ccai_null","ccai_null","ccai_null"],["ccai_null","ccai_null","ccai_null","ccai_null"],["2","2","2","2"])
# dedupe(["325030591_9487190","325027589_9487190","331666833_6733095","325027590_9487190","325027588_9487190","325030592_9487190","325030593_9487190","325030594_9487190","332467231_9745991","325027587_9487190"],["christian","christina janell","christian","christian jr","christensen janaya","christensen janaya","christina janell","christian jr","christian","christian"],["tonido","tonido","tonido","tonido","tonido","tonido","tonido","tonido","tonido","tonido"],["1985-12-05","2014-01-12","1987-11-18","2015-12-22","2009-12-11","2009-10-11","2014-01-12","2015-07-22","1997-11-28","1985-12-05"],["tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","ccai_null","tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","ccai_null","tonidojinkyc@gmail.com"],["tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","ccai_null","tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","tonidojinkyc@gmail.com","ccai_null","tonidojinkyc@gmail.com"],["1","2","1","1","2","2","2","1","1","1"])

UDF called
UDF called 2
!!!!!!!!!!!!!!
[[2], [3, 4], [1]]
          FirstName LastName DateOfBirth  ProvisionalPrimaryKey Gender
0           candice     iyog  1973-11-16                      2      2
1  candice jennifer     iyog  1973-11-16                      3      2
2  candice jennifer     iyog   ccai_null                      1      2
UDF called 3
UDF called 4
[[3, 2], [1, 3], [2], [3, 4], [1]]
UDF called 4
UDF called 5


,return,firstname,dob,gender
0,c4ca4238a0b923820dcc509a6f75849b,candice jennifer,ccai_null,2
1,c4ca4238a0b923820dcc509a6f75849b,candice,1973-11-16,2
2,c4ca4238a0b923820dcc509a6f75849b,candice jennifer,1973-11-16,2
3,c4ca4238a0b923820dcc509a6f75849b,candice jennifer,1973-11-16,2


In [4]:
4*0.72

2.88

In [47]:
import re

str1 = "ashrithanil.jr"

# detect ' jr' or '.jr' in str1
# if re.search(r' jr|\.jr', str1):
    # replace ' jr' or '.jr' with ' son junior'
re.sub(r' jr|\.jr', ' son junior'+str1, str1)


'ashrithanil son juniorashrithanil.jr'

In [35]:
import pandas
import recordlinkage as rl
import recordlinkage
from recordlinkage.datasets import load_krebsregister
from recordlinkage.datasets import load_febrl1

In [48]:
dfA["idk"] = None
dfA["idk"].value_counts(normalize=True).iloc[0]

IndexError: single positional indexer is out-of-bounds

In [46]:
dfA = load_febrl1()
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(dfA)

In [5]:
candidate_links

MultiIndex([( 'rec-122-org',   'rec-223-org'),
            ( 'rec-373-org',   'rec-223-org'),
            ( 'rec-373-org',   'rec-122-org'),
            ('rec-10-dup-0',   'rec-223-org'),
            ('rec-10-dup-0',   'rec-122-org'),
            ('rec-10-dup-0',   'rec-373-org'),
            ( 'rec-227-org',   'rec-223-org'),
            ( 'rec-227-org',   'rec-122-org'),
            ( 'rec-227-org',   'rec-373-org'),
            ( 'rec-227-org',  'rec-10-dup-0'),
            ...
            ( 'rec-212-org',   'rec-205-org'),
            ( 'rec-212-org', 'rec-416-dup-0'),
            ( 'rec-212-org',   'rec-330-org'),
            ( 'rec-212-org',   'rec-385-org'),
            ( 'rec-212-org', 'rec-113-dup-0'),
            ( 'rec-212-org',   'rec-327-org'),
            ( 'rec-212-org', 'rec-188-dup-0'),
            ( 'rec-212-org', 'rec-334-dup-0'),
            ( 'rec-212-org', 'rec-469-dup-0'),
            ( 'rec-212-org', 'rec-350-dup-0')],
           names=['rec_id_1', 'rec_id_2'], 

In [7]:
krebs_X, krebs_true_links = load_krebsregister(missing_values=0)

In [6]:
krebs_X

,,cmp_firstname1,cmp_firstname2,cmp_lastname1,cmp_lastname2,cmp_sex,cmp_birthday,cmp_birthmonth,cmp_birthyear,cmp_zipcode
id1,id2,,,,,,,,,
22161,38467,1.000000,0.0,0.142857,0.0,1,0.0,1.0,0.0,0.0
38713,75352,0.000000,0.0,0.571429,0.0,1,0.0,0.0,0.0,0.0
13699,32825,0.166667,0.0,0.000000,0.0,0,1.0,1.0,1.0,0.0
22709,37682,0.285714,0.0,1.000000,0.0,1,0.0,0.0,0.0,0.0
2342,69060,0.250000,0.0,0.125000,0.0,1,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
52124,53629,1.000000,0.0,0.285714,0.0,1,0.0,0.0,1.0,0.0
30007,76846,0.750000,0.0,0.000000,0.0,1,1.0,0.0,0.0,0.0
50546,59461,0.750000,0.0,0.000000,0.0,1,0.0,1.0,0.0,0.0


In [8]:
kmeans = rl.KMeansClassifier()
result_kmeans = kmeans.fit_predict(krebs_X)
len(result_kmeans)

371525

In [11]:
# matches_list = result_kmeans.index.tolist()
matches_list = [list(x) for x in result_kmeans]
matches_list


[[13699, 32825],
 [2342, 69060],
 [89874, 89876],
 [87277, 98883],
 [38193, 99682],
 [39162, 97240],
 [50528, 68975],
 [35508, 44391],
 [44402, 47954],
 [19252, 32997],
 [12845, 61200],
 [45053, 76552],
 [53328, 53998],
 [27195, 91296],
 [56712, 70632],
 [24547, 52523],
 [32514, 97083],
 [24389, 50715],
 [50414, 50760],
 [7195, 59150],
 [15232, 37451],
 [52194, 90516],
 [35105, 61892],
 [10501, 47560],
 [12015, 26232],
 [54866, 96487],
 [5787, 50628],
 [24303, 62660],
 [75194, 87935],
 [18169, 41897],
 [53818, 61934],
 [81282, 81538],
 [79126, 84983],
 [28308, 39158],
 [4859, 6014],
 [72598, 90113],
 [28628, 75659],
 [24751, 42943],
 [6756, 27323],
 [40350, 83715],
 [47461, 51944],
 [69599, 97276],
 [75394, 92002],
 [30804, 85959],
 [20827, 85506],
 [12283, 50801],
 [23323, 27823],
 [41881, 88290],
 [2182, 53673],
 [9759, 35367],
 [68841, 82540],
 [40441, 80589],
 [72552, 91133],
 [31059, 72216],
 [36598, 79032],
 [4389, 87430],
 [36583, 60576],
 [19451, 45450],
 [6920, 7547],
 [17498,

In [5]:
krebs_true_links

MultiIndex([(89874, 89876),
            (79126, 84983),
            (40350, 83715),
            (75394, 92002),
            (23323, 27823),
            (31059, 72216),
            (28464, 69899),
            (33613, 64971),
            (23546, 27978),
            (29922, 46075),
            ...
            (62697, 62705),
            (82751, 82753),
            (45892, 59324),
            (33619, 40411),
            (28562, 43893),
            (26162, 27408),
            (71973, 71976),
            (23925, 29112),
            (57711, 99655),
            (18795, 41061)],
           names=['id1', 'id2'], length=20931)

In [1]:
# initiate pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# import ceja
from pyspark.sql import types
from pyspark import SparkContext, SparkConf

spark = SparkSession.builder.appName('customer360.ai').getOrCreate()

# initiate spark context
sc = spark.sparkContext
spark.conf.set("spark.sql.mapKeyDedupPolicy", "LAST_WIN")


Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m


23/02/19 10:21:40 WARN Utils: Your hostname, mindgraphin-cebucdppysp-7sykz44qliu resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface ceth0)
23/02/19 10:21:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/19 10:21:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
    select id, concat_ws(',', collect_list(arrivalstation) over (partition by id order by journeynumber)) as arrivalstation
    from df
""")


In [19]:
# create a dataframe with columns arrivalstation,journeynumber,id
df = spark.createDataFrame([
    ("A", "1", "1"),
    ("B", "2", "1"),
    ("A", "2", "2"),
    ("B", "3", "3"),
    ("A", "2", "3"),
    ("A", "4", "4"),
    ("A", "5", "5"),
], ["arrivalstation", "journeynumber", "id"])
# create a temp table
df.createOrReplaceTempView("df")

# concat arrivalstation based on ascending order of journeynumber grouped by id. keep the last value
# df = spark.sql("""
#     select id, concat_ws(',', collect_list(arrivalstation) over (partition by id order by journeynumber)) as arrivalstation
#     from df

# """)
# "select * from all_passengerjourneyleg order by PassengerID, ModifiedUTC desc", "SELECT PassengerID, CONCAT_WS('_',COLLECT_LIST(UnitDesignator)) AS UnitDesignator FROM all_passengerjourneyleg GROUP BY PassengerID"
df = spark.sql("""
    select * from df order by id, journeynumber desc
""")
df.createOrReplaceTempView("df")
df = spark.sql("""
    select id, concat_ws(',', collect_list(arrivalstation)) as arrivalstation
    from df group by id
""")
# df.createOrReplaceTempView("df")
# # drop duplicates using sql
# df = spark.sql("""
#     select id, arrivalstation
#     from df
#     group by id, arrivalstation
# """)

df.show()


+---+--------------+
| id|arrivalstation|
+---+--------------+
|  3|           B,A|
|  5|             A|
|  1|           B,A|
|  4|             A|
|  2|             A|
+---+--------------+



In [109]:
# create a dataframe with pFirstName column
df = spark.createDataFrame([
    ("mr john doe",),
    ("ms jane doe",),
    ("dr john doe",),
    ("rev john doe",),
    ("dr CENON JR",),
    ("dr. vina",)
], ["pFirstName"])
df = df.withColumn("pFirstName", F.regexp_replace(F.col("pFirstName"), "\.", " "))

df = df.withColumn("pFirstName", F.regexp_replace(F.col("pFirstName"), "^(mr|ms|dr|rev|prof|sir|madam|miss|mrs|st)", ""))
# remove any leading or trailing spaces
df = df.withColumn("pFirstName", F.trim(F.col("pFirstName")))
df.show()

+----------+
|pFirstName|
+----------+
|  john doe|
|  jane doe|
|  john doe|
|  john doe|
|  CENON JR|
|      vina|
+----------+



In [3]:
df = spark.read.parquet("dedup_input.snappy.parquet")
# print df columns
df.printSchema()

root
 |-- ProvisionalPrimaryKey: string (nullable = true)
 |-- PassengerID: long (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- BookingID: long (nullable = true)
 |-- TravelSoloOrGroup: string (nullable = true)
 |-- PersonID: long (nullable = true)
 |-- IsRegistered: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- HonorificPrefix: string (nullable = true)
 |-- HonorificSuffix: string (nullable = true)
 |-- EmailAddress: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- ResidentCountry: string (nullable = true)
 |-- CustomerNumber: string (nullable = true)
 |-- PassportNumber: s

In [4]:
df = spark.read.parquet("dedup_input.snappy.parquet")
# filter out TBAA, PAX, FNAME, FirstName from FirstName column
# filter out TBAA, PAX, TBA, LNAME, LastName from LastName column

df = df.select("ProvisionalPrimaryKey","FirstName", "LastName", "DateOfBirth", "Gender","EmailAddress", "Phone", "PassportNumber")
firstname_filters = ["TBAA","PAX", "FNAME", "xxxx", "FirstName","api","anonymous","xxx","cebitcc"]
lastname_filters = ["TBAA", "PAX", "TBA", "LNAME", "xxxx"]


# filter out firstname_filters from FirstName column using like
for filter in firstname_filters:
    df = df.filter(~F.col("FirstName").like("%{}%".format(filter)))

# filter out lastname_filters from LastName column using like
for filter in lastname_filters:
    df = df.filter(~F.col("LastName").like("%{}%".format(filter)))

# make FNU in FirstName column as null
df = df.withColumn("FirstName", F.when(F.col("FirstName") == "FNU", None).otherwise(F.col("FirstName")))
# make DateOfBirth as null if first four elements is less than 1947
df = df.withColumn("DateOfBirth", F.when(F.col("DateOfBirth").substr(0, 4) < "1947", None).otherwise(F.col("DateOfBirth")))
df = df.withColumn("DateOfBirth", F.when(F.col("DateOfBirth").substr(0, 4) > "3000", None).otherwise(F.col("DateOfBirth")))
df = df.withColumn("DateOfBirth", F.when(F.col("DateOfBirth").like("9999%"), None).otherwise(F.col("DateOfBirth")))

# convert FirstName, LastName to lower case, remove spaces and special characters
df = df.withColumn("FirstName", F.lower(F.regexp_replace(F.col("FirstName"), "[^a-zA-Z0-9]", "")))
df = df.withColumn("LastName", F.lower(F.regexp_replace(F.col("LastName"), "[^a-zA-Z0-9]", "")))


df = df.withColumn("pFirstName", ceja.metaphone(F.col("FirstName"))).withColumn("pLastName", ceja.metaphone(F.col("LastName")))

# df.groupBy("pFirstName", "pLastName", "DateOfBirth").agg(F.collect_list("FirstName").alias("FirstName"),F.collect_list("LastName").alias("LastName"),  F.count("*").alias("count"), F.collect_list("ProvisionalPrimaryKey").alias("ProvisionalPrimaryKey"), F.first("ProvisionalPrimaryKey").alias("ProvisionalPrimaryKey"))


In [7]:
df.show(10)

+---------------------+---------------+--------+-----------+------+------------+-----+--------------+----------+---------+
|ProvisionalPrimaryKey|FirstName      |LastName|DateOfBirth|Gender|EmailAddress|Phone|PassportNumber|pFirstName|pLastName|
+---------------------+---------------+--------+-----------+------+------------+-----+--------------+----------+---------+
|286708890_4842825    |david          |duran   |1999-10-19 |2     |null        |null |null          |TFT       |TRN      |
|286734204_9          |michaeledward  |pedernal|1994-07-07 |1     |null        |null |null          |MXLTWRT   |PTRNL    |
|286862577_978975     |johnwilson     |rosco   |2000-12-19 |1     |null        |null |null          |JNWLSN    |RSK      |
|288560130_9          |jonito         |silverio|1960-12-15 |1     |null        |null |null          |JNT       |SLFR     |
|296181813_9          |henjiben       |leganti |1996-05-16 |1     |null        |null |null          |HNJBN     |LKNT     |
|297279364_9    

In [9]:
import splink.spark.spark_comparison_library as cl
settings = {
    # "ProvisionalPrimaryKey","FirstName", "LastName", "DateOfBirth", "Gender","EmailAddress", "Phone", "PassportNumber"
    "link_type": "dedupe_only",
    "comparisons": [
        cl.jaro_winkler_at_thresholds("FirstName", 0.8),
        cl.jaro_winkler_at_thresholds("LastName", 0.8),
        cl.levenshtein_at_thresholds("DateOfBirth"),
        cl.levenshtein_at_thresholds("Gender"),
        cl.levenshtein_at_thresholds("EmailAddress"),
        cl.exact_match("Phone"),
        cl.exact_match("PassportNumber"),
    ],
    "unique_id_column_name": "ProvisionalPrimaryKey",
    "blocking_rules_to_generate_predictions": [
        "l.pFirstName = r.pFirstName and l.pLastName = r.pLastName",
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}
from splink.spark.spark_linker import SparkLinker
linker = SparkLinker(df, settings)





/workspace/.pyenv_mirror/user/current/lib/python3.8/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [10]:
deterministic_rules = [
    "l.FirstName = r.FirstName and levenshtein(r.DateOfBirth, l.DateOfBirth) <= 1",
    "l.LastName = r.LastName and levenshtein(r.DateOfBirth, l.DateOfBirth) <= 1",
    "l.pFirstName = r.pFirstName and levenshtein(r.LastName, l.LastName) <= 2",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)


Probability two random records match is estimated to be  3.13e-05.              
This means that amongst all possible pairwise record comparisons, one in 31,974.14 are expected to match.  With 746,969,226 total possible comparisons, we expect a total of around 23,361.67 matching pairs


In [ ]:
linker.estimate_u_using_random_sampling(target_rows=df.count())

In [ ]:
linker.estimate_m_from_label_column("PassportNumber")
df_predictions = linker.predict(threshold_match_probability=0.2)
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predictions, threshold_match_probability=0.7)

# END

In [42]:
df.groupBy("pFirstName", "pLastName", "DateOfBirth").agg(F.collect_list("FirstName").alias("FirstName"),F.collect_list("LastName").alias("LastName"),  F.count("*").alias("count")).sort(F.col("pFirstName"), F.col("pLastName"), F.col("DateOfBirth")).write.parquet("dedup_output.snappy.parquet", mode="overwrite")

In [11]:
# groupyby pFirstName, last_name and get count sort by count desc
df.groupBy("FirstName", "LastName", "DateOfBirth", "Gender").count().sort(F.desc("count")).show()

+-----------------+-------------+-----------+------+-----+
|        FirstName|     LastName|DateOfBirth|Gender|count|
+-----------------+-------------+-----------+------+-----+
|          Ferraci|  Sanvictores| 1995-07-22|     1|  512|
|Princess Robvenyl|        Laygo| 2007-01-15|     2|  256|
|          Sherwin|      Andrada| 1979-07-07|     1|  145|
|          Rosalia|      Hilomen| 1962-03-28|     2|  128|
|         GENEROSE|       SALIVA| 1991-05-02|     2|  128|
|            Jelle|    Hellemans| 1985-12-25|     1|  128|
|          Fabrice|      VICHARD| 1973-08-09|     1|   72|
|             Amor|   Candelaria| 1976-08-21|     2|   64|
|         Younjung|          Kim| 1979-06-30|     2|   64|
|    Lilia Binaoro|      Docusin| 1965-03-14|     2|   64|
|         Alastair|         Reed| 1978-12-07|     1|   64|
|      Dianne Rose|    Marutsuka| 1993-02-14|     2|   48|
|        Estrelita|       Lontok| 1969-11-06|     2|   32|
|            Jakob|       Runkel| 1954-08-25|     1|   3

In [5]:
df = spark.read.parquet("firstlastname1.snappy.parquet")
# get rows with DateOfBirth array length greater than 1
df.filter(F.size(F.col("DateOfBirth")) > 1).show()
# save to csv

+--------------------+
|         DateOfBirth|
+--------------------+
|[1994-08-15, 1994...|
|[1993-12-11, 1993...|
|[1990-07-11, 1990...|
|[1965-03-22, 1965...|
|[2001-12-01, 2001...|
|[2002-07-19, 2002...|
|[1979-12-19, 1979...|
|[1986-07-12, 1996...|
|[1994-08-29, 1994...|
|[1990-11-19, 1990...|
|[1996-06-10, 1996...|
|[2000-10-25, 2000...|
|[1994-06-12, 1994...|
|[1956-11-22, 1956...|
|[1996-10-08, 1996...|
|[1984-09-17, 1984...|
|[1978-02-07, 1978...|
|[1992-02-27, 1992...|
|[1993-03-16, 1993...|
|[1997-08-26, 1997...|
+--------------------+
only showing top 20 rows



In [43]:
import pandas as pd
df = pd.read_parquet("dedup_output.snappy.parquet")
# get rows with DateOfBirth array length greater than 1
df[df["count"] > 1].sort_values(by="count", ascending=False).to_csv("dedup_output.csv", index=False)

In [11]:
from pyspark.sql import Window

from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
# import ArrayType, StringType, IntegerType, FloatType, DoubleType, BooleanType, DateType, TimestampType
from pyspark.sql.types import ArrayType, StringType, IntegerType, FloatType, DoubleType, BooleanType, DateType, TimestampType
import pandas as pd
data = [
    (1, "Jacob","1985-02-12"),
    # ("Jacob",None),
    (2, "kristine joy","1985-02-12"),
    (3, "Jacob","1984-02-12"),

    (4, "Shim","1985-02-12"),
    (5, "Shim","1985-03-12"),
    (6, None,"1985-02-12")
]
df111 = spark.createDataFrame(data, ["id","word","DateOfBirth"])
# actual_df = df111.withColumn("word_match_rating_codex", ceja.metaphone(F.col("word")))
# actual_df.show()

# fill null with 9999-12-31
df111 = df111.withColumn("DateOfBirth", F.when(F.col("DateOfBirth").isNull(), "9999-12-31").otherwise(F.col("DateOfBirth")))

df111 = df111.groupBy("word").agg(F.collect_list("DateOfBirth").alias("DateOfBirth"), F.count("*").alias("count"), F.collect_list("id").alias("id"))

def get_dates_array(id_arr,date_arr):
    # create pandas dataframe
    df = pd.DataFrame({"id":id_arr, "DateOfBirth":date_arr})
    


get_array_udf = F.udf(get_dates_array, ArrayType(StringType()))
import hashlib
def pcolumn_hashing(word):
    return hashlib.md5(str(word).encode("utf-8")).hexdigest()
df111 = df111.withColumn("group_hash" , F.udf(pcolumn_hashing, StringType())(F.col("word")))


# df111 = df111.withColumn("DateOfBirths", get_array_udf(F.col("id"),F.col("DateOfBirth")))
# df111.show()

# new rows for each DateOfBirths
df111.show()

df111 = df111.withColumn("tmp",F.arrays_zip("id","DateOfBirth")).withColumn("DateOfBirth", F.explode("tmp")).select("word","DateOfBirth.*","group_hash")
df111 = df111.withColumn("word_hash", F.sha2(F.col("word"),224))

df111.show()



+------------+--------------------+-----+------+--------------------+
|        word|         DateOfBirth|count|    id|          group_hash|
+------------+--------------------+-----+------+--------------------+
|       Jacob|[1985-02-12, 1984...|    2|[1, 3]|9adb41f2a66506c7e...|
|kristine joy|        [1985-02-12]|    1|   [2]|b27e9401f726976da...|
|        Shim|[1985-02-12, 1985...|    2|[4, 5]|d99ff19e6e035e726...|
|        null|        [1985-02-12]|    1|   [6]|6adf97f83acf6453d...|
+------------+--------------------+-----+------+--------------------+

+------------+---+-----------+--------------------+--------------------+
|        word| id|DateOfBirth|          group_hash|           word_hash|
+------------+---+-----------+--------------------+--------------------+
|       Jacob|  1| 1985-02-12|9adb41f2a66506c7e...|5d441e2af696ff14e...|
|       Jacob|  3| 1984-02-12|9adb41f2a66506c7e...|5d441e2af696ff14e...|
|kristine joy|  2| 1985-02-12|b27e9401f726976da...|b365922904457d705...|
|

In [12]:
import hashlib
hashlib.md5(str(None).encode("utf-8")).hexdigest()

'6adf97f83acf6453d4a6a4b1070f3754'

In [56]:
import re 

def check(email):  
    # (?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])

    regex = """(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""
    if(re.search(regex,email)):  
        print("Valid Email")  
        return email
    else:  
        print("Invalid Email")
        return None  
check("jacob@kk.com")
check("anil.turaga@mind-graph.com")
check("anil..com")

Valid Email
Valid Email
Invalid Email


In [110]:
data = [
    (1, "Jacob","jacob@kk.com"),
    # ("Jacob",None),
    (2, "kristine joy","krisjoy@"),
    (3, "Jacob.Jr","jc.com"),

    (4, "Shim","shim@nfenf"),
    (5, "Shim","shim@nfenf.ai"),
    (6, None,"wodms6364naver.com")
]
df = spark.createDataFrame(data, ["id","FirstName","EmailAddress"])
# regex on email address to validate
df = df.withColumn("EmailAddress",
                    F.when(
                        F.col("EmailAddress").rlike("^(.+@.+\\..+)(; .+@.+\\..+)*$"),
                        F.col("EmailAddress")
                    ).otherwise("ccai_null")
                  )
df = df.withColumn("FirstName", F.lower(F.regexp_replace(F.col("FirstName"), "[^a-zA-Z0-9 ]", "")))

df = df.withColumn("pFirstName", F.split(F.col("FirstName")," ").getItem(0))


df.show()

+---+------------+-------------+----------+
| id|   FirstName| EmailAddress|pFirstName|
+---+------------+-------------+----------+
|  1|       jacob| jacob@kk.com|     jacob|
|  2|kristine joy|    ccai_null|  kristine|
|  3|     jacobjr|    ccai_null|   jacobjr|
|  4|        shim|    ccai_null|      shim|
|  5|        shim|shim@nfenf.ai|      shim|
|  6|        null|    ccai_null|      null|
+---+------------+-------------+----------+



# dedupe training prep

In [3]:
import pandas as pd
from pyspark.sql import types as T
from abydos.phonetic import *

sm = SpanishMetaphone()


df_ne = spark.read.parquet("dedupe_training.snappy.parquet")

# df_ne.count()
# df_ne.show()
df_ne = df_ne.sort(F.desc("dob_count"))
# get dob_count column as list
max_dob_count = df_ne.select("dob_count").rdd.flatMap(lambda x: x).collect()
df_ne = df_ne.filter(F.col("dob_count") == max_dob_count[3])
# 3, 4
# df_ne.show()

# explode FirstName, LastName, DateOfBirth, EmailAddress, Phone, ProvisionalPrimaryKey and hash using array zip
df_ne = df_ne.withColumn("tmp",F.arrays_zip("FirstName","LastName","DateOfBirth","EmailAddress","Phone","ProvisionalPrimaryKey", "Gender")).withColumn("FirstName", F.explode("tmp")).select("FirstName.*").drop("tmp")
# save as single csv file

df_ne.toPandas().to_csv("dedupe_inference00.csv", index=False)

df_ne.show()

AnalysisException: Column 'dob_count' does not exist. Did you mean one of the following? [Phone, FirstName, LastName, group_hash, pLastName, DateOfBirth, pFirstName, EmailAddress, passenger_hash, ProvisionalPrimaryKey];
'Sort ['dob_count DESC NULLS LAST], true
+- Relation [pFirstName#0,pLastName#1,group_hash#2,passenger_hash#3,ProvisionalPrimaryKey#4,FirstName#5,LastName#6,DateOfBirth#7,EmailAddress#8,Phone#9] parquet


In [26]:
import jellyfish
print(jellyfish.jaro_winkler_similarity("joanne", "jane"))
print(jellyfish.levenshtein_distance("joanne", "jane"))
print(jellyfish.damerau_levenshtein_distance("joanne", "jane"))

0.8999999999999999
2
2


In [8]:
from abydos.phonetic import *
sm = SpanishMetaphone() 
print(sm.encode("zara vee"))
print(sm.encode("Adrian"))

ZRV
ADRN


In [14]:
print("dob: ", 659708/10664831, "\n email: ",9300295/10664831,"\n phone: ", 9397188/10664831)

dob:  0.06185827042172539 
 email:  0.8720527310746884 
 phone:  0.8811380133449841


In [107]:
data = [
    (1, "june","andreda"),
    (2, "adelnino","asd"),
    (7, "sherwin","andreda"),
    (3, "sherwin","andredas"),
    (4, "andreda","sherwin"),
    (5, "jeaann","effefef"),
    (6, None,"1985-02-12")
]
df111 = spark.createDataFrame(data, ["id","word","DateOfBirth"])

df111.createOrReplaceTempView("df111")
# filter out sherwin andreda if in the same row
df111 = spark.sql("select * from df111 where word not like '%sherwin%' or DateOfBirth not like '%andreda%'")

# df111 = df111.withColumn("pword", ceja.match_rating_codex(F.col("word")))
df111.show()

NameError: name 'spark' is not defined

In [28]:
firstname_filters = ["TBAA","PAX", "FNAME", "xxxx", "FirstName","api","anonymous","xxx","cebitcc"]
lastname_filters = ["TBAA", "PAX", "TBA", "LNAME", "xxxx"]

# query = "FirstName not in {} and LastName not in {}".format(tuple(firstname_filters), tuple(lastname_filters))
query = "FirstName not like '%{}%' and LastName not like '%{}%'".format("%' and FirstName not like '%".join(firstname_filters), "%' and LastName not like '%".join(lastname_filters))
query = "FirstName not in ('{}')".format("', '".join(firstname_filters))
query = "FirstName not like '%{}%'".format("%' and FirstName not like '%".join(firstname_filters))

query

"FirstName not like '%TBAA%' and FirstName not like '%PAX%' and FirstName not like '%FNAME%' and FirstName not like '%xxxx%' and FirstName not like '%FirstName%' and FirstName not like '%api%' and FirstName not like '%anonymous%' and FirstName not like '%xxx%' and FirstName not like '%cebitcc%'"

In [31]:
data = [
    ("joanne", "joan"),
    ("there", "their"),
    ("luisa", "bruna"),
    (None, None)
]
df = spark.createDataFrame(data, ["word1", "word2"])
actual_df = df.withColumn("match_rating_comparison", ceja.match_rating_comparison(F.col("word1"), F.col("word2")))
actual_df.show()

+------+-----+-----------------------+
| word1|word2|match_rating_comparison|
+------+-----+-----------------------+
|joanne| joan|                   true|
| there|their|                   true|
| luisa|bruna|                  false|
|  null| null|                   null|
+------+-----+-----------------------+



In [64]:
import pandas as pd
df_v1 = pd.read_parquet("valid01.snappy.parquet")
df_v1["GetGoID"].value_counts()

xxxx                19108
1003436852              3
8220001000412093        3
8110002006814277        2
2008006039              2
                    ...  
8330006000006875        1
8110002007687300        1
8110002001258020        1
8110001015699554        1
2031299189              1
Name: GetGoID, Length: 2384, dtype: int64

In [38]:
df_too_many2 = spark.read.parquet("too_many2.snappy.parquet")
df_too_many2.show()

+----------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+----------+---------+
|pFirstName|pLastName|           FirstName|            LastName|         DateOfBirth|        EmailAddress|               Phone|ProvisionalPrimaryKey|              Gender|group_hash|dob_count|
+----------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+----------+---------+
|       NTV|        A|[ntba, ntba, ntba...|[aaa, aaa, aaa, a...|[ccai_null, ccai_...|[ccai_null, ccai_...|[ccai_null, ccai_...| [325188817_969563...|[2, 2, 2, 1, 2, 1...|     NTV A|     5877|
+----------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+----------+---------+



In [34]:
df_too_many = spark.read.parquet("too_many")
df_too_many.show()
df_too_many = df_too_many.withColumn("tmp",F.arrays_zip("FirstName","LastName","DateOfBirth","EmailAddress","Phone","ProvisionalPrimaryKey", "Gender")).withColumn("FirstName", F.explode("tmp")).select("FirstName.*").drop("tmp")
df_too_many.show()


+----------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+------------+---------+
|pFirstName|pLastName|           FirstName|            LastName|         DateOfBirth|        EmailAddress|               Phone|ProvisionalPrimaryKey|              Gender|  group_hash|dob_count|
+----------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+------------+---------+
|    ESERUN|    ANDRD|[sherwin, sherwin...|[andrada, andrada...|[1979-07-07, 1979...|[ccai_null, ccai_...|[ccai_null, ccai_...| [325704604_969559...|[1, 1, 1, 1, 1, 1...|ESERUN ANDRD|    44892|
|      ZVTX|   KMNDZN|[cebitcc, cebitcc...|[commandcenter, c...|[1993-08-04, 1993...|[ccai_null, ccai_...|[ccai_null, ccai_...| [329018251_948719...|[1, 1, 1, 1, 1, 1...| ZVTX KMNDZN|     9527|
|       NTV|        A|[ntba1, 

+---------+--------+-----------+------------+---------+---------------------+------+
|FirstName|LastName|DateOfBirth|EmailAddress|    Phone|ProvisionalPrimaryKey|Gender|
+---------+--------+-----------+------------+---------+---------------------+------+
|  sherwin| andrada| 1979-07-07|   ccai_null|ccai_null|    325704604_9695599|     1|
|  sherwin| andrada| 1979-07-07|   ccai_null|ccai_null|    325735634_9695599|     1|
|  sherwin| andrada| 1979-07-07|   ccai_null|ccai_null|    325762815_9695599|     1|
|  sherwin| andrada| 1979-07-07|   ccai_null|ccai_null|    325888135_9695599|     1|
|  sherwin| andrada| 1979-07-07|   ccai_null|ccai_null|    325917018_9695599|     1|
|  sherwin| andrada| 1979-07-07|   ccai_null|ccai_null|    325949488_9695599|     1|
|  sherwin| andrada| 1979-07-07|   ccai_null|ccai_null|    325956385_9695599|     1|
|  sherwin| andrada| 1979-07-07|   ccai_null|ccai_null|    325970796_9695599|     1|
|  sherwin| andrada| 1979-07-07|   ccai_null|ccai_null|    325971

In [35]:
df_too_many.count()

61449

In [36]:
firstname_filters = ["TBAA","PAX", "FNAME", "xxxx", "FirstName","api","anonymous","xxx","cebitcc", "ntba1"]
lastname_filters = ["TBAA", "PAX", "TBA", "LNAME", "xxxx"]
# log count
# LOGGER.info("ccai Total count: {}".format(df.count()))
# sql query to filter out rows with FirstName and LastName in firstname_filters and lastname_filters
df_too_many.createOrReplaceTempView("df_too_many")
firstname_query = "FirstName not like '%{}%'".format("%' and FirstName not like '%".join(firstname_filters))
lastname_query = "LastName not like '%{}%'".format("%' and LastName not like '%".join(lastname_filters))
df_too_many = spark.sql("select * from df_too_many where {} and {}".format(firstname_query, lastname_query))


In [37]:
df_too_many.count()

50769

In [46]:

import pandas as pd
df_c3 = pd.read_parquet("dedupe_training.snappy.parquet")
df_c3

,pFirstName,pLastName,group_hash,passenger_hash,ProvisionalPrimaryKey,FirstName,LastName,DateOfBirth,EmailAddress,Phone
0,JSSKMY,KRLS,67f9b835186dbb16344ce3a1a6dde8df,51af0cb92199f24b0cf08ea80de11f61,334436816_9487190,jessacamille,carlos,1996-07-06,ccai_null,ccai_null
1,JSSKMY,KRLS,67f9b835186dbb16344ce3a1a6dde8df,51af0cb92199f24b0cf08ea80de11f61,334417198_9487190,jessacamille,carlos,1996-07-06,ccai_null,ccai_null
2,SSN,MSD,5e732cf16983a3e3cf33a92fcdae2b70,a69b9a72ec28f0247d6d987ba8694dd6,332961670_7802390,susan,mesde,1950-07-17,rolandomesde@yahoo.com,09228734476
3,SSN,MSD,5e732cf16983a3e3cf33a92fcdae2b70,a69b9a72ec28f0247d6d987ba8694dd6,330759547_7802390,susan,mesde,1950-07-17,ccai_null,ccai_null
4,SSN,MSD,5e732cf16983a3e3cf33a92fcdae2b70,a69b9a72ec28f0247d6d987ba8694dd6,330759880_7802390,susan,mesde,1950-07-17,sheryl@mesde.net,63-9991685211
...,...,...,...,...,...,...,...,...,...,...
86705,ESENMR,KYNG,0e393a3916d074df8c4398aa4baba6c4,cef305064aaad9c0807c2ef28d28e12d,328186786_7785796,sheenamarie,callanga,1995-03-02,ccai_null,ccai_null
86706,ESENMR,KYNG,0e393a3916d074df8c4398aa4baba6c4,cef305064aaad9c0807c2ef28d28e12d,328218629_9487190,sheenamarie,callanga,1995-03-02,callangasheenamarie@gmail.com,63-9774806543
86707,VRN,KGLNGN,5ae07cd35d7acc361da679dc43561215,efe418891955b13e00e13e8fc9972525,327152379_2857900,brian,cagalingan,1987-11-29,ccai_null,ccai_null
86708,VRN,KGLNGN,5ae07cd35d7acc361da679dc43561215,efe418891955b13e00e13e8fc9972525,325996429_2857900,brian,cagalingan,1987-11-29,ccai_null,ccai_null


In [4]:
arr = [1,2,3]
str(arr)

'[1, 2, 3]'

In [13]:
from pyspark.sql import Row

eDF = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])
eDF.show()
# eDF = eDF.select(F.posexplode(eDF.mapfield))
eDF.select(F.posexplode(eDF.intlist)).show()


+---+---------+--------+
|  a|  intlist|mapfield|
+---+---------+--------+
|  1|[1, 2, 3]|{a -> b}|
+---+---------+--------+

+---+---+
|pos|col|
+---+---+
|  0|  1|
|  1|  2|
|  2|  3|
+---+---+



In [13]:
import pandas as pd
df_c3 = spark.read.parquet("part-00000-566f0b8e-2610-437a-a16e-e45f824bf640-c000.snappy.parquet")

In [15]:
df_c3.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+------------------+--------------------+--------------------+--------------------+---------+--------------------+
|           FirstName|            LastName|         DateOfBirth|        EmailAddress|               Phone|ProvisionalPrimaryKey|            Gender|         PassengerID|            PersonID|          group_hash|dob_count|      passenger_hash|
+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+------------------+--------------------+--------------------+--------------------+---------+--------------------+
|                [ae]|            [heuang]|        [1966-10-19]|         [ccai_null]|         [ccai_null]|  [334268770_7548238]|               [2]|         [334268770]|           [7548238]|7bc7882971b16bbef...|        1|[ec10e390cf1a22af...|
|               [haa]|          

In [16]:
collect_list_columns = ["FirstName", "LastName", "DateOfBirth", "EmailAddress", "Phone", "ProvisionalPrimaryKey", "Gender", "PassengerID", "PersonID"]
# explode collect_list_columns using arrays_zip
df_c3.withColumn("tmp",F.arrays_zip(*collect_list_columns)).withColumn("FirstName", F.explode("tmp")).select("FirstName.*").drop("tmp").show()

+---------+----------+-----------+--------------------+-------------+---------------------+------+-----------+--------+
|FirstName|  LastName|DateOfBirth|        EmailAddress|        Phone|ProvisionalPrimaryKey|Gender|PassengerID|PersonID|
+---------+----------+-----------+--------------------+-------------+---------------------+------+-----------+--------+
|       ae|    heuang| 1966-10-19|           ccai_null|    ccai_null|    334268770_7548238|     2|  334268770| 7548238|
|      haa|        hk| 1992-09-16|           ccai_null|    ccai_null|    326276643_9487190|     2|  326276643| 9487190|
|      aoi|   carmona| 1993-01-14|           ccai_null|    ccai_null|    331885814_9487190|     2|  331885814| 9487190|
|      aoi|   carmona| 1993-01-14|           ccai_null|    ccai_null|    331894375_9487190|     2|  331894375| 9487190|
|      aoi|   carmona| 1993-01-14|           ccai_null|    ccai_null|    331883552_9487190|     2|  331883552| 9487190|
|      aoi|   carmona| 1993-01-14|      

In [3]:
import pandas as pd
df_profiles = pd.read_parquet("profiles.snappy.parquet")
df_profiles

,ProvisionalPrimaryKey,PassengerID,Revenue,BookingID,TravelSoloOrGroup,PersonID,IsRegistered,FirstName,MiddleName,LastName,...,AdditionalContactEmail,AdditionalContactWorkPhone,AdditionalContactHomePhone,AdditionalContactCity,AdditionalContactState,AdditionalContactZipcode,AdditionalContactCountryCode,AdditionalContactCustomerNumber,PrimaryContactStreetAddress,AdditionalContactStreetAddress
0,117524961_61169,117524961,15.01,89601971,Solo,61169,Yes,xxxx,xxxx,xxxx,...,None,None,None,None,None,None,None,None,"xxxx, xxxx, xxxx",
1,121927604_648206,121927604,2562.72,92286806,Solo,648206,Yes,xxxx,xxxx,xxxx,...,None,None,None,None,None,None,None,None,"xxxx, xxxx, xxxx",
2,122166468_776445,122166468,7580.82,92435551,Solo,776445,Yes,xxxx,xxxx,xxxx,...,None,None,None,None,None,None,None,None,"xxxx, xxxx, xxxx",
3,122494154_782840,122494154,0.00,92641496,Solo,782840,Yes,xxxx,xxxx,xxxx,...,None,None,None,None,None,None,None,None,"xxxx, xxxx, xxxx",
4,123610684_410177,123610684,660.04,93362023,Solo,410177,Yes,xxxx,xxxx,xxxx,...,None,None,None,None,None,None,None,None,"xxxx, xxxx, xxxx",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147052,335316826_9487190,335316826,2660.64,204681516,Solo,9487190,No,Nimfa Ligaya,None,Mimay,...,None,None,None,None,None,None,None,None,,
147053,335317333_586937,335317333,5691.00,204681834,Solo,586937,No,DAVID PAUL,None,ADONA,...,None,None,None,None,None,None,None,None,,
147054,335317461_9487190,335317461,10104.36,204681902,Solo,9487190,No,Jaqueline,None,Arisme,...,None,None,None,None,None,None,None,None,,
147055,335318159_9487190,335318159,2213.76,204682312,Group,9487190,No,Felix Josef,None,Grave,...,None,None,None,None,None,None,None,None,,


In [4]:
len(df_profiles.columns)

84

In [8]:
import textdistance
textdistance.jaro_winkler.similarity("Maria Cielo Candice", "MariaCielo")

0.9176470588235295

In [111]:
import pandas as pd
df_temp = pd.read_parquet("temp.snappy.parquet")
# df_temp = df_temp[0:10000]


In [112]:
df_temp

,FirstName,LastName,DateOfBirth,EmailAddress,Phone,ProvisionalPrimaryKey,Gender,PassengerID,PersonID,passenger_hash,group_hash
0,ha,lucam,1994-03-07,ccai_null,ccai_null,332658193_9487190,2,332658193,9487190,7495114aabcbf11221d41ff00501d95c,8a0cb9e87350982122c1c374a89ccaf7
1,ha,lucam,1994-03-07,ccai_null,ccai_null,332656894_9487190,2,332656894,9487190,7495114aabcbf11221d41ff00501d95c,8a0cb9e87350982122c1c374a89ccaf7
2,ha,lucam,1994-12-07,ccai_null,ccai_null,332136135_9487190,2,332136135,9487190,979a84dff00601570f88116304e67a63,8a0cb9e87350982122c1c374a89ccaf7
3,ha,lucam,1994-03-07,ccai_null,ccai_null,332657577_9487190,2,332657577,9487190,7495114aabcbf11221d41ff00501d95c,8a0cb9e87350982122c1c374a89ccaf7
4,ha,lucam,1994-03-07,ccai_null,ccai_null,332508753_9487190,2,332508753,9487190,7495114aabcbf11221d41ff00501d95c,8a0cb9e87350982122c1c374a89ccaf7
...,...,...,...,...,...,...,...,...,...,...,...
109826,cecille giellian,rivero,1991-12-26,mmarequez@dha.gov.ae,971-505166751,333371500_9487190,2,333371500,9487190,7159b7a91382e6803afd9d57f37b264d,0512e1f07bd926a22303ba356e192630
109827,cecille giellian,rivero,1991-12-26,ccai_null,ccai_null,333371180_9487190,2,333371180,9487190,7159b7a91382e6803afd9d57f37b264d,0512e1f07bd926a22303ba356e192630
109828,cecille marie luisi,castro,1984-07-16,ccai_null,ccai_null,325817443_10377017,2,325817443,10377017,919d9e312e7d3e254dba468e8d19d37e,6b4d978b791b6e0b4b82067526cf7865
109829,cecille anne,moscoso,1985-05-27,ccai_null,ccai_null,331547554_9487190,2,331547554,9487190,9fc65c1c9fd685427336a0b47e6e4baa,5fa799efb1d8637498fa388f3879ee29


In [114]:
df_temp.columns

Index(['FirstName', 'LastName', 'DateOfBirth', 'EmailAddress', 'Phone',
       'ProvisionalPrimaryKey', 'Gender', 'PassengerID', 'PersonID',
       'passenger_hash', 'group_hash'],
      dtype='object')

In [104]:
l1 = [1,22,3]
l2 = [4,5,6]
l3 = [8,9,10]
l4,l5,l6 = zip(*sorted(zip(l1,l2,l3)))


In [106]:
l4,l5,l6

((1, 3, 22), (4, 6, 5), (8, 10, 9))

In [93]:
df_temp_gp = df_temp.groupby(["FirstName","LastName", "DateOfBirth"])
print(df_temp.shape)
list_of_lists = df_temp_gp["ProvisionalPrimaryKey"].apply(list).to_list()
df_temp_gp = df_temp_gp.first().reset_index()
print(df_temp_gp.shape)

(10000, 11)
(5540, 11)


In [92]:
df_temp.shape, df_temp_gp.shape

((10000, 11), (5605, 11))

In [ ]:
df_temp_gp.set_index('ProvisionalPrimaryKey')

In [69]:
len(list_of_lists)

5605

In [87]:
import recordlinkage
indexer = recordlinkage.Index()
# indexer.full()
indexer.add(recordlinkage.index.SortedNeighbourhood(left_on="FirstName",window=501))
# indexer = recordlinkage.SortedNeighbourhoodIndex(
#     'FirstName', window=100000
# )
candidate_links = indexer.index(df_temp)
len(candidate_links)


6240760

In [88]:
import recordlinkage
indexer = recordlinkage.Index()
indexer.full()
# indexer.add(recordlinkage.index.SortedNeighbourhood(left_on="FirstName",window=11))
# indexer = recordlinkage.SortedNeighbourhoodIndex(
#     'FirstName', window=100000
# )
candidate_links = indexer.index(df_temp)
len(candidate_links)


49995000

In [89]:
6240760/49995000

0.12482768276827683

In [55]:
import networkx
from networkx.algorithms.components.connected import connected_components

def to_graph(l):
    G = networkx.Graph()
    for part in l:
        # each sublist is a bunch of nodes
        G.add_nodes_from(part)
        # it also imlies a number of edges:
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    """ 
        treat `l` as a Graph and returns it's edges 
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
    """
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    
matches_list = [[1,2,3], [3,4,8], [5,6,7]]
G = to_graph(matches_list)
cluster = [list(x) for x in list(connected_components(G))]
cluster

[[1, 2, 3, 4, 8], [5, 6, 7]]

In [2]:
from unidecode import unidecode

# Example 1: Basic Latin characters
result1 = unidecode("Hello, world!")
print(result1)  # Output: Hello, world!

# Example 2: Latin characters with diacritics
result2 = unidecode("Café au Lait")
print(result2)  # Output: Cafe au Lait

# Example 3: Cyrillic characters
result3 = unidecode("Привет, мир!")
print(result3)  # Output: Privet, mir!

# Example 4: Greek characters
result4 = unidecode("Γεια σας")
print(result4)  # Output: Geia sas

# Example 5: Japanese characters (Hiragana)
result5 = unidecode("こんにちは")
print(result5)  # Output: konnichiha

# Example 6: Chinese characters (Simplified)
result6 = unidecode("你好，世界！")
print(result6)  # Output: Ni Hao , Shi Jie !


Hello, world!
Cafe au Lait
Privet, mir!
Geia sas
konnichiha
Ni Hao ,Shi Jie !
